## Assignment 1 - IO

Author: David Henning (worked with Natasha Watkinga, Chandni Raja and Jonathan Kowarski)

In [1]:
import numpy as np
import pandas as pd
from statsmodels.api import OLS, tsa
from statsmodels.sandbox.regression.gmm import IV2SLS

In [2]:
d = pd.read_csv("PS1_Data/OTC_Data.csv", sep='\t')
d[d.duplicated(subset=['store', 'week', 'brand'], keep=False)] # No duplicates at store-week-brand level

,store,week,brand,sales_,count,price_,prom_,cost_


In [3]:
d_demographic = pd.read_csv("PS1_Data/OTCDemographics.csv", sep='\t')

In [4]:
d.sort_values(by=['store', 'week', 'brand'], inplace=True)

In [5]:
#Try to reproduce summary stat table from the problem set
d['revenue'] = d['sales_'] * d['price_']
d['revenue_brand'] = d.groupby(['brand'])['revenue'].transform('sum')
d['sales_brand'] = d.groupby(['brand'])['sales_'].transform('sum')
d['price_brand'] = d.groupby(['brand'])['price_'].transform('mean')
d['cost_brand'] = d.groupby(['brand'])['cost_'].transform('mean')
d['branded'] = d['brand']
d.loc[d['branded'] <= 9, 'branded'] = 1
d.loc[d['branded'] > 9, 'branded'] = 0
d_summary = d[['brand', 'revenue_brand', 'sales_brand', 'price_brand', 'cost_brand']]
d_summary = d_summary[:11]
d_summary['revenue_tot'] = d_summary['revenue_brand'].sum()
d_summary['share'] = d_summary['revenue_brand'] / d_summary['revenue_tot'] * 0.62

In [6]:
d_instruments = pd.read_csv("PS1_Data/OTCDataInstruments.csv", sep='\t')

### Problem 1

Consider utility function for good $j$ in store-week $t$ for consumer $i$:
    $$ u_{ijt} = X_{jt}\beta + \alpha p_{jt} + \xi_{jt} + \epsilon_{jt} $$
   

###### (1) Estimate this model using OLS with price and promotion as product characteristics

In [7]:
# calculate the shares
df = d[['store', 'week', 'brand', 'prom_', 'price_', 'cost_', 'sales_']]
df['sales_store_week'] = df.groupby(['store', 'week'])['sales_'].transform('sum')
df['s'] = df['sales_'] / df['sales_store_week'] * 0.62 # All of the brands have a total market share of 62%

<ipython-input-7-2521c2f842fd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sales_store_week'] = df.groupby(['store', 'week'])['sales_'].transform('sum')
<ipython-input-7-2521c2f842fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['s'] = df['sales_'] / df['sales_store_week'] * 0.62 # All of the brands have a total market share of 62%


In [8]:
# Transform variables for regression, recall outside options share is 0.38
s_0 = 0.38
df['δ'] = np.log(df['s']) - np.log(s_0)
df['constant'] = 1

<ipython-input-8-1cb6825e2c25>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['δ'] = np.log(df['s']) - np.log(s_0)
<ipython-input-8-1cb6825e2c25>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['constant'] = 1


In [9]:
# Run OLS regression
x = df.drop(['δ', 'store', 'week', 'brand',  'cost_', 'sales_', 'sales_store_week', 's',], 1)
y = df['δ']
results1 = OLS(y, x).fit()
model1_α = results1.params['price_']
#print(results2.summary())

###### (2) Estimate this model using OLS with price and promotion as product characteristics and brand dummies

In [10]:
#Add dummies to dataframe
b_dummies = pd.get_dummies(df['brand'])
df_dummies = df.merge(b_dummies, left_index = True, right_index = True)

In [11]:
x = df_dummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'sales_', 'sales_store_week', 's',], 1)
y = df_dummies['δ']
results2 = OLS(y, x).fit()
model2_α = results2.params['price_']
#print(results2.summary())

###### (3) Estimate this model using OLS with price and promotion as product characteristics and store-brand dummies

In [12]:
store_brand = df['store'].astype('str') + '_' + df['brand'].astype('str') 
bs_dummies = pd.get_dummies(store_brand)
df_bsdummies = df.merge(bs_dummies, left_index = True, right_index = True)

In [13]:
x = df_bsdummies.drop(['δ', 'store', 's', 'brand', 'week', 'cost_', 'sales_', 'sales_store_week'], 1)
y = df_bsdummies['δ']

In [14]:
results3 = OLS(y, x).fit()
model3_α = results3.params['price_']
#print(results3.summary())

###### (4) Estimate the models of 1, 2 and 3 using wholesale cost as an instrument.

In [15]:
# Run IV regression for first problem
x = df.drop(['δ', 'store', 'week', 'brand',  'cost_', 'sales_', 'sales_store_week', 's',], 1)
z = df.drop(['δ', 'store', 'week', 'brand', 'sales_', 'price_', 'sales_store_week', 's',], 1)
y = df['δ']
results4 = IV2SLS(y, x, instrument=z).fit()
#print(results4.summary())

In [16]:
# Run IV regression for second problem
x = df_dummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'sales_', 'sales_store_week', 's',], 1)
z = df_dummies.drop(['δ', 'store', 'week', 'brand', 'sales_', 'price_', 'sales_store_week', 's',], 1)
y = df['δ']
results4 = IV2SLS(y, x, z).fit()
#print(results4.summary())

In [17]:
# Run IV regression for third problem
x = df_bsdummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'sales_', 'sales_store_week', 's',], 1)
z = df_bsdummies.drop(['δ', 'store', 'week', 'brand', 'price_', 'sales_', 'sales_store_week', 's',], 1)
y = df['δ']
results4 = IV2SLS(y, x, z).fit()
#print(results4.summary())

###### (5) Estimate the models of 1, 2 and 3 using the Hausman instrument (average price in other markets).

In [18]:
#Calculate hausman instrument, and add to all the datasets
grouped = df.groupby(['brand', 'week'])
n = grouped['price_'].transform('count')
mean = grouped['price_'].transform('mean')
df['price_iv'] = (mean*n - df['price_'])/(n-1)
df_dummies['price_iv'] = (mean*n - df['price_'])/(n-1)
df_bsdummies['price_iv'] = (mean*n - df['price_'])/(n-1)
df.sort_values(by=['brand', 'week', 'store'], inplace=True)

<ipython-input-18-3314831a76a0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_iv'] = (mean*n - df['price_'])/(n-1)
<ipython-input-18-3314831a76a0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by=['brand', 'week', 'store'], inplace=True)


In [19]:
#Run the IV with  the Hausman instrument for (1)
x = df.drop(['δ', 'store', 'week', 'brand',  'cost_', 'price_iv', 'sales_', 'sales_store_week', 's',], 1)
z = df.drop(['δ', 'store', 'week', 'brand',  'cost_', 'price_', 'sales_', 'sales_store_week', 's',], 1)
y = df['δ']
results5 = IV2SLS(y, x, z).fit()
#print(results5.summary())

In [20]:
#Run the IV with  the Hausman instrument for (2)
x = df_dummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'price_iv', 'sales_', 'sales_store_week', 's',], 1)
z = df_dummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'price_', 'sales_', 'sales_store_week', 's',], 1)
y = df_dummies['δ']
results5 = IV2SLS(y, x, z).fit()
#print(results5.summary())

In [21]:
#Run the IV with  the Hausman instrument for (3)
x = df_bsdummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'price_iv', 'sales_', 'sales_store_week', 's',], 1)
z = df_bsdummies.drop(['δ', 'store', 'week', 'brand',  'cost_', 'price_', 'sales_', 'sales_store_week', 's',], 1)
y = df_bsdummies['δ']
results5 = IV2SLS(y, x, z).fit()
#print(results5.summary())

###### (6) Using the analytic formula for elasticity of the logit model, compute the mean own-price elasticities for all brand in the market using the estimates in 1, 2 and 3. Do these results make sense?

In [22]:
d_summary['model1_η'] = -model1_α * d_summary['price_brand'] * (1 - d_summary['share'])
d_summary['model2_η'] = -model2_α * d_summary['price_brand'] * (1 - d_summary['share'])
d_summary['model3_η'] = -model3_α * d_summary['price_brand'] * (1 - d_summary['share'])
#d_summary

## Problem 2

Consider utility function for good $j$ in store-week $t$ for consumer $i$:
    $$ u_{ijt} = X_{jt}\beta + \beta_{ib}B_{jt}\text{(Branded Product)} + \alpha_i p_{jt} + \xi_{jt} + \epsilon_{ijt} $$
Where:
$$\beta_{ib} = \sigma_B\nu_i$$
$$\alpha_i = \alpha + \sigma_I I_i$$

Hence $$ u_{ijt} = X_{jt}\beta + \sigma_B\nu_iB_{jt}\text{(Branded Product)} + (\alpha + \sigma_I I_i) p_{jt} + \xi_{jt} + \epsilon_{ijt} $$

Or, rearranging $$ u_{ijt} = X_{jt}\beta + \alpha p_{jt} + \xi_{jt} + \sigma_B\nu_iB_{jt}\text{(Branded Product)} + \sigma_I I_i p_{jt} + \epsilon_{ijt} $$


In [23]:
d_blp = pd.merge(d, d_demographic)

np.random.seed(1)
η = np.random.normal(scale=1, size=ns) 

#Put everything into arrays
X = d['prom_']
X['constant'] = 1
X = X.values
p = d['price_']
B = d['branded']
I = d_blp.iloc[:, d_blp.columns.str.startswith('hhincome')].values

#Setting the frequencies
ns = I.shape[1]

#Starting values for parameters
β = 0.01
α = 0.01
σ_B = 0.01
σ_I = 0.01
#ξ = 0.01
I
s_hat = np.zeros(I.shape[0])
s_hat
ns

NameError: name 'ns' is not defined

In [ ]:
#Calculate predicted shares
for i in range(1, ns):
    d_blp['numerator'] = α * d_blp['price_'] + σ_B * η[i-1] * d_blp['branded'] + σ_I * d_blp['hhincome' + str(i)] * d_blp['price_']
    denominator =  d_blp.groupby(['store', 'week'])['numerator'].transform('sum') + 1
    numerator = d_blp['numerator'] 
    s_hat += numerator / denominator * (1/ns)

d_blp['s_hat'] = s_hat
s_hat